In [1]:
#Checks/installs plotly in order to view data visualizations
if 'google.colab' in str(get_ipython()):
    !pip install plotly==4.14.3

In [2]:
#Imports pandas, numpy, and plotly for the database
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go

For this case study, I selected the player batting stats of the 2020 MLB season. I selected this dataset because I wanted to see how the players performed in this shortened 60-game season. Also, I am very passionate about baseball and wanted to see if I coould learn something new from investigating the data further. From this, I wanted to find out who the best hitters in baseball were last there while figuring out who the worst hitters were that were given a lot of at-bats. Also, I wanted to figure out who the best and worst teams were at producing runs compared to their divisions and leagues.

In [3]:
#Imports MLB Batting stats for the 2020 season broken down into players playing for their specific teams.
#***This means players that changed teams during the season will have multiple data entries but this is the only way to ensure team batting stats will be accurate.***
df = pd.read_csv('https://github.com/vankley2/MLB-Analyst/raw/main/2020_MLB_Batting.csv')

In [4]:
#Displays number of different player entries and different data columns for batting
df.shape

(696, 31)

In [5]:
#Displays columns of the dataset and how many entries per column
#Nonessential info(There are 696 entries but this drops to 651 for batting average(BA) because some players didn't record an at-bat and were just pinch runners or
# defensive replacements. But, this number goes back up to 652 for on-base percentage(OBP). This means a player recorded a plate appearance but didn't record
# an at-bat because BA, SLG, OPS, and OPS+ are calculated using at-bats while OBP only requires a plate appearance. And, this annomaly means that the player
# didn't have an at-bat all season but had a plate appearance and got on-base in order to show up here. And that player is Michael Lorenzen who had one PA
# and got hit by a pitch to get on base. Here is the PA: https://baseballsavant.mlb.com/sporty-videos?playId=20b7b7c1-b16b-43e0-aef2-218bad3cba80
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rk           696 non-null    int64  
 1   Name         696 non-null    object 
 2   Age          696 non-null    int64  
 3   Tm           696 non-null    object 
 4   Lg           696 non-null    object 
 5   G            696 non-null    int64  
 6   PA           696 non-null    int64  
 7   AB           696 non-null    int64  
 8   R            696 non-null    int64  
 9   H            696 non-null    int64  
 10  2B           696 non-null    int64  
 11  3B           696 non-null    int64  
 12  HR           696 non-null    int64  
 13  RBI          696 non-null    int64  
 14  SB           696 non-null    int64  
 15  CS           696 non-null    int64  
 16  BB           696 non-null    int64  
 17  SO           696 non-null    int64  
 18  BA           651 non-null    float64
 19  OBP     

In [6]:
df.head()

,Rk,Name,Age,Tm,Lg,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary,Player ID
0,1,José Abreu,33,CHW,AL,60,262,240,43,76,15,0,19,60,0,0,18,59,0.317,0.370,0.617,0.987,165.0,148,10,3,0,1,1,*3/D,abreujo02
1,2,Ronald Acuna Jr.,22,ATL,NL,46,202,160,46,40,11,0,14,29,8,1,38,60,0.250,0.406,0.581,0.987,152.0,93,3,4,0,0,2,*89,acunaro01
2,3,Willy Adames,24,TBR,AL,54,205,185,29,48,15,1,8,23,2,1,20,74,0.259,0.332,0.481,0.813,127.0,89,4,0,0,0,0,*6/H,adamewi01
3,4,Austin Adams,29,SDP,NL,1,1,1,0,0,0,0,0,0,0,0,0,1,0.000,0.000,0.000,0.000,-100.0,0,0,0,0,0,0,/1,adamsau02
4,5,Matt Adams,31,ATL,NL,16,51,49,4,9,2,0,2,9,0,0,2,18,0.184,0.216,0.347,0.563,43.0,17,3,0,0,0,0,D/H3,adamsma01


In [7]:
df.tail()

,Rk,Name,Age,Tm,Lg,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary,Player ID
691,692,Mike Yastrzemski,29,SFG,NL,54,225,192,39,57,14,4,10,35,2,1,30,55,0.297,0.400,0.568,0.968,165.0,109,2,3,0,0,2,*98/7H,yastrmi01
692,693,Christian Yelich,28,MIL,NL,58,247,200,39,41,7,1,12,22,4,2,46,76,0.205,0.356,0.430,0.786,111.0,86,4,1,0,0,2,*7/D,yelicch01
693,694,Andrew Young,26,ARI,NL,12,34,26,3,5,2,0,1,4,0,0,5,10,0.192,0.382,0.385,0.767,107.0,10,0,3,0,0,0,/4D5H7,youngan02
694,695,Bradley Zimmer,27,CLE,AL,20,50,37,3,6,0,0,1,3,2,1,7,14,0.162,0.360,0.243,0.603,69.0,9,2,5,0,1,0,/789H,zimmebr01
695,696,Mike Zunino,29,TBR,AL,28,84,75,8,11,4,0,4,10,0,0,6,37,0.147,0.238,0.360,0.598,67.0,27,0,3,0,0,0,2,zuninmi01


In [8]:
df.sample(5)

,Rk,Name,Age,Tm,Lg,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary,Player ID
27,28,Eddy Alvarez,30,MIA,NL,12,41,37,6,7,1,0,0,2,2,0,3,16,0.189,0.268,0.216,0.485,34.0,8,0,1,0,0,0,/45H6,alvared01
260,261,Billy Hamilton,29,CHC,NL,14,11,10,6,3,0,0,1,1,3,1,1,4,0.300,0.364,0.600,0.964,158.0,6,0,0,0,0,0,8/HD,hamilbi02
136,137,Nelson Cruz,39,MIN,AL,53,214,185,33,56,6,0,16,33,0,0,25,58,0.303,0.397,0.595,0.992,173.0,110,8,4,0,0,5,*D/H,cruzne02
512,513,J.T. Realmuto,29,PHI,NL,47,195,173,33,46,6,0,11,32,4,1,16,48,0.266,0.349,0.491,0.840,126.0,85,3,6,0,0,0,2/D3H,realmjt01
182,183,Phil Ervin,27,CIN,NL,19,42,35,5,3,0,0,0,0,1,0,6,8,0.086,0.238,0.086,0.324,-10.0,3,1,1,0,0,0,H7/8D9,ervinph01


In [9]:
#Drops duplicates of players that would be used for team stats and only takes the combined total of each player over the season
#(The database is set up so the total of a players' stats is listed above their stats for specific teams which means "drop duplicates" works here)
df_players = df.drop_duplicates('Player ID', keep='first')
num_dups = df.shape[0] - df_players.shape[0]
print(f"There were {num_dups} duplicates of players in this dataset.")

There were 78 duplicates of players in this dataset.


In [10]:
#Displays players sorted by their Home Run totals during the season
df_hr = df_players.sort_values('HR',ascending=False)
df_hr

,Rk,Name,Age,Tm,Lg,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary,Player ID
664,665,Luke Voit,29,NYY,AL,56,234,213,41,59,5,0,22,52,0,0,17,54,0.277,0.338,0.610,0.948,157.0,130,4,3,0,1,0,*3/DH,voitlu01
0,1,José Abreu,33,CHW,AL,60,262,240,43,76,15,0,19,60,0,0,18,59,0.317,0.370,0.617,0.987,165.0,148,10,3,0,1,1,*3/D,abreujo02
470,471,Marcell Ozuna,29,ATL,NL,60,267,228,38,77,14,0,18,56,0,0,38,60,0.338,0.431,0.636,1.067,172.0,145,3,0,0,1,3,D7/9,ozunama01
610,611,Fernando Tatis Jr.,21,SDP,NL,59,257,224,50,62,11,2,17,45,11,3,27,61,0.277,0.366,0.571,0.937,157.0,128,6,5,0,1,1,*6/D,tatisfe02
630,631,Mike Trout,28,LAA,AL,53,241,199,41,56,9,2,17,46,1,1,35,56,0.281,0.390,0.603,0.993,164.0,120,1,3,0,4,4,*8/D,troutmi01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,456,Brian O'Grady,28,TBR,AL,2,5,5,2,2,1,0,0,0,1,0,0,1,0.400,0.400,0.600,1.000,179.0,3,0,0,0,0,0,/387,ogradbr01
458,459,Joseph Odom,28,SEA,AL,18,44,39,2,5,0,0,0,2,0,0,4,20,0.128,0.209,0.128,0.338,-2.0,5,1,0,1,0,0,2/H,odomjo01
150,151,Austin Dean,26,STL,NL,3,7,4,1,1,1,0,0,0,0,0,3,2,0.250,0.571,0.500,1.071,202.0,2,0,0,0,0,0,/73H9,deanau01
461,462,Jared Oliva,24,PIT,NL,6,16,16,0,3,0,0,0,0,1,0,0,6,0.188,0.188,0.188,0.375,3.0,3,0,0,0,0,0,/78H,olivaja01


In [11]:
#Displays the Home Run Leader for the MLB regular season
hr_count = df_hr['HR'].max()
hr_leader = df_hr[df_hr['HR'] == hr_count]
hr_leader

,Rk,Name,Age,Tm,Lg,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary,Player ID
664,665,Luke Voit,29,NYY,AL,56,234,213,41,59,5,0,22,52,0,0,17,54,0.277,0.338,0.61,0.948,157.0,130,4,3,0,1,0,*3/DH,voitlu01


In [12]:
#Displays the number of runs scored by each team
#***TOT is not an actual team and has been removed from this dataset
df_runs = df.groupby('Tm').agg({'R':'sum'})
df_runs = df_runs.drop('TOT')
df_runs

,R
Tm,
ARI,269
ATL,348
BAL,274
BOS,292
CHC,265
CHW,306
CIN,243
CLE,248
COL,275


In [13]:
#Displays only the hitters qualified for the Batting Title in 2020
#(A batter needs 3.1 Plate Appearances per team game to qualify which is 186 PA's in 2020 because it was a 60-game season but would be 502 in a regular 162-game season)
df_qualified = df_players.loc[df["PA"] >= 186]
df_qualified = df_qualified.sort_values('BA',ascending=False)
df_qualified

,Rk,Name,Age,Tm,Lg,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary,Player ID
350,351,DJ LeMahieu,31,NYY,AL,50,216,195,41,71,10,2,10,27,3,0,18,21,0.364,0.421,0.590,1.011,178.0,115,3,2,0,1,0,435/HD,lemahdj01
589,590,Juan Soto,21,WSN,NL,47,196,154,39,54,14,0,13,37,6,2,41,28,0.351,0.490,0.695,1.185,217.0,107,1,1,0,0,12,*7/9D,sotoju01
208,209,Freddie Freeman,30,ATL,NL,60,262,214,51,73,23,1,13,53,2,0,45,37,0.341,0.462,0.640,1.102,182.0,137,6,3,0,0,7,*3/DH,freemfr01
470,471,Marcell Ozuna,29,ATL,NL,60,267,228,38,77,14,0,18,56,0,0,38,60,0.338,0.431,0.636,1.067,172.0,145,3,0,0,1,3,D7/9,ozunama01
635,636,Trea Turner,27,WSN,NL,59,259,233,46,78,15,4,12,41,12,4,22,36,0.335,0.394,0.588,0.982,162.0,137,5,2,0,2,0,*6,turnetr01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,431,Max Muncy,29,LAD,NL,58,248,203,36,39,4,0,12,27,1,0,39,60,0.192,0.331,0.389,0.720,97.0,79,3,4,0,2,2,354/D,muncyma01
520,521,Bryan Reynolds,25,PIT,NL,55,208,185,24,35,6,2,7,19,1,1,21,57,0.189,0.275,0.357,0.632,70.0,66,2,1,0,0,0,*78/HD,reynobr01
563,564,Kyle Schwarber,27,CHC,NL,59,224,191,30,36,6,0,11,24,1,0,30,66,0.188,0.308,0.393,0.701,90.0,75,3,3,0,0,1,*7/DH,schwaky01
212,213,Joey Gallo,26,TEX,AL,57,226,193,23,35,8,0,10,26,2,0,29,79,0.181,0.301,0.378,0.679,83.0,73,0,4,0,0,2,*9/DH8,gallojo01


In [14]:
num_qualified = df_qualified.shape[0]
print(f"There were {num_qualified} qualified hitters in the 2020 season for a Batting Title.")

There were 142 qualified hitters in the 2020 season for a Batting Title.


In [15]:
fig1 = px.scatter(df_qualified, x='SO', y='HR', title='Home Runs vs. Strikeouts in 2020')
fig1

In [16]:
fig2 = px.box(df_qualified, x='OPS+', orientation='h', title='On-Base Plus Slugging % in 2020')
fig2

In [17]:
fig3 = px.histogram(df_qualified,x='BA',title='Batting Average Distribution', nbins=50)
fig3

In [18]:
fig4 = px.scatter_3d(df_qualified, x='OBP',y='SLG',z='HR',title = 'Home Runs, On-Base, and Slugging Percentage 3D Scatter Plot',color='Lg', template='plotly_dark')
fig4

In [19]:
fig5 = px.sunburst(df,path=['Lg','Tm'],title = 'Scoring Output by League and Team', values = 'R',width=800,height=800)
fig5

From the findings above, we can see that both the Home Run Leader and Batting Title Champion were on the same team last year as Luke Voit and DJ LeMahieu had great seasons last year. This contributed to the New York Yankees(NYY) success last year as they scored the most runs last year in the American League(AL). Also, Evan White had a rough season last year as he was last in the whole league among qualified hitters in Batting Average(BA). Hopefully this turns around for him as he is only 24 years old. The outlier on the OPS+ graph with an outlandish 217 OPS+ is Juan Soto who played only 47 games last year. This might be a small sample size because the last time this feat was accomplished was in 2004 over an 162-game season. The National League(NL) outscored the American League(AL) which is rare because the AL usually has a designated hitter(DH) who can bat for the pitcher while the NL doesn't. However, the MLB instituted a universal DH this season in order to deal with the extended layoff due to COVID-19.